# 아래 코드의 목적 : For Factoring
Factoring은 회계상 매출채권(Account Receivables)을 제3자에게 판매하여, 즉각적인 유동성 확보 및 회계부채감소의 효과를 기대한다.<br/>
그래서 제3자에서 Factoring을 계약하기 위해서는 과거 및 현재 채권의 거래안전성을 확인해야 한다.<br/>
즉 각 개별거래처의 매출채권(Receivables)이 계약대로 이행(입금)이 되었거나, 이행되지 않는 경우 얼마나 지연되었는가를 확인해야 한다.<br/><br/>
이 절차는 매우 중요하다. 왜냐하면 제3자와 Factoring 계약시, 계약조건 및 수수료 등 비용에 큰 변동이 있을 수 있기 때문이다.(당신이라면 대금결제를 4개월 이상 지연하는 매출채권을 매입하겠는다?)<br/><br/>
아래 코드는 각 기간/ID별 매출의 질(입금이 이행되었는가? 아니면 어느정도 지연되었는가?)을 판단하기 위함이다. (매우 많은 계산을 요구한다)<br/><br/>

* 참고로 모 통신사들에서 Factoring을 위한, 매출채권의 코드를 만드는데 약 2개월이 소요되었다. (간단하다면 간단하고, 복잡하다면 매우 복잡하다)

* https://en.wikipedia.org/wiki/Factoring_(finance)

# 분석에 대한 기본설명
만약 개똥이네라는 곳에서 2017.01에 100만원 매출이 발생했다. 개똥이네는 1달 매출액은 다음달 말일에 입금해야한다<br/><br/>
매출 : 100만원(2017.01)<br/>
입금 : 20만원(2017.02) / 10만원(2017.03) / 50만원(2017.04) / 20만원(2017.05)<br/><br/>

100만원 매출 중, 정상적으로 입금된 금액은 20만원 밖에 안된다. 심지어 최종입금완료는 3개월이 지연되었다. (나쁜놈이다!)<br/>
아래 코드는 개똥이네 같은 자료를 분석하기 위해서이다.<br/><br/>

* 만약 손으로 상기자료를 만든다면? ID 3만개 * 31개월 = 그만 알아보자...
* 실제 자료를 분석한다면, 개똥이네같이 깔끔히 떨어지는 자료를 거의 없다. 코드를 분석하면서, 예외부분을 설명하겠다

In [1]:
import pandas as pd
import numpy as np
import calendar

from pandas import DataFrame, Series
from datetime import datetime, date, time, timedelta

## 자료읽기
UTF-8 으로 인코딩된 CSV 자료

In [37]:
df_raw = pd.read_csv('./csv/date.csv')

## 변수설명
id : 고객코드<br/>
date : 날짜(월말기준)<br/>
base : 기초잔고<br/>
rev : 매출<br/>
dep : 입금<br/>
end : 기말잔고<br/>
dueDateMonth : 매출발생 이후, 입금일 (단위는 1달 기준) // ie: 2.0 - 2달 뒤 입금

In [38]:
df_raw['date'] = pd.to_datetime(df_raw['date']) # date를 날짜형으로 변경

In [39]:
df_raw.head()

,id,date,base,rev,dep,end,dueDateMonth
0,522781082,2016-01-31,7636293.0,5872124.0,7643334.0,5865083.0,1.0
1,310211001,2016-01-31,0.0,54335535.0,54335535.0,0.0,0.0
2,310641001,2016-01-31,-2894476.0,7594650.0,7500000.0,-2799826.0,0.0
3,522781088,2016-01-31,9033249.0,6852563.0,9033249.0,6852563.0,1.0
4,315531001,2016-01-31,-12708.0,10915690.0,0.0,10902982.0,1.0


## 매월 말 날짜만 추출
직관적으로 이해하기는 어렵겠지만, Factoring은 ID별 월매출액의 입금을 기간별로 분해야한다.


In [40]:
dates = df_raw.date.unique()

In [41]:
dates

array(['2016-01-31T00:00:00.000000000', '2016-02-29T00:00:00.000000000',
       '2016-03-31T00:00:00.000000000', '2016-04-30T00:00:00.000000000',
       '2016-05-31T00:00:00.000000000', '2016-06-30T00:00:00.000000000',
       '2016-07-31T00:00:00.000000000', '2016-08-31T00:00:00.000000000',
       '2016-09-30T00:00:00.000000000', '2016-10-31T00:00:00.000000000',
       '2016-11-30T00:00:00.000000000', '2016-12-31T00:00:00.000000000',
       '2017-01-31T00:00:00.000000000', '2017-02-28T00:00:00.000000000',
       '2017-03-31T00:00:00.000000000', '2017-04-30T00:00:00.000000000',
       '2017-05-31T00:00:00.000000000', '2017-06-30T00:00:00.000000000',
       '2017-07-31T00:00:00.000000000', '2017-08-31T00:00:00.000000000',
       '2017-09-30T00:00:00.000000000', '2017-10-31T00:00:00.000000000',
       '2017-11-30T00:00:00.000000000', '2017-12-31T00:00:00.000000000',
       '2018-01-31T00:00:00.000000000', '2018-02-28T00:00:00.000000000',
       '2018-03-31T00:00:00.000000000', '2018-04-30

In [42]:
start = datetime.now()

## 1달 단위로 id를 추출 후, 해당 id 매출(revenue)의 회입기간을 분석
예전에 작성한 코드로 함수로 쪼개야 함에도 불구하고, 귀찮아서 그냥 작성된 코드를 정리만 했음.<br/>
매출채권에 대한 이해가 낮은 경우, 상당히 코드 해석이 어려울 수 있음

In [43]:
recap = DataFrame()

for date1 in dates:
    
    ## 1년 동안의 매출채권자료만 추출
    date2 = date1 + np.timedelta64(350, 'D')
    df = df_raw[(date1 <= df_raw['date']) & (df_raw['date'] < date2)]
    
    ## 메모리 관리를 위하여, date1에 존재하는 id만 추출
    customers = df.loc[df['date'] == date1, 'id'].unique()
    df = df[df['id'].isin(customers)]
    
    ## id와 날짜를 기준으로 정렬
    df = df.sort_values(['id', 'date'])
    
    ## 매출이 발생한 날짜를 기준으로 index를 추출
    df_index = df.groupby(['id'])['date'].apply(lambda x: x.idxmin())
    ## 매출이 발생한 날자를 표시
    df['date_rev'] = df.loc[df_index, 'date']; df['date_rev'] = df['date_rev'].fillna(method='ffill')

    ## 매출이 발생하기 전, 기초잔고가 음수(-)인 경우 -> 입금액에 기초잔고를 반영하고, 기초잔고를 0으로 만든다
    df.loc[df_index, 'dep'] = df.loc[df_index, 'dep'].where(df.loc[df_index, 'base'] >= 0
                                                            , df.loc[df_index, 'dep'] - df.loc[df_index, 'base'])
    df.loc[df_index, 'base'] = df.loc[df_index, 'base'].where(df.loc[df_index, 'base'] >= 0, 0)

    ## 상기반영된 기초잔고를 base_temp라는 열에 반영한다
    df['base_temp'] = df.loc[df_index, 'base']; df['base_temp'] = df['base_temp'].fillna(method='ffill')

    ## id별 입금액의 누적합계를 구한 후, 차분(differentiate)한다
    df['dep_cum'] = df.groupby(['id'])['dep'].transform(lambda x: np.cumsum(x))
    df['dep_cum_diff'] = df.groupby(['id'])['dep_cum'].apply(lambda x: x.diff().fillna(0))

    ## 반영된 기초잔고(base_temp)를 입금에 반영하기 위한 중간단계
    ## (기준월에 이전에 발생된 매출을 입금에서 제외하면, 기준월 매출발생의 입금월을 확인할 수 있기 때문)
    # 실제 입금에 기초잔고액 입금을 반영하기 위한 중간 단계로 dep_add라를 열을 생성
    df['dep_add'] = df['dep'].where(df['base_temp'] >= df['dep_cum'], df['base_temp'] - (df['dep_cum'] - df['dep_cum_diff'])) 
    
    ## dep_add가 음수(-)로 발생된 인덱스를 찾은 후, 매출기준월(date_rev)와 날짜(date)가 같은 경우 dep_add에 기초잔고(base)금액을 반영한다
    i = df.loc[df['dep_add'] < 0].index
    df.loc[i, 'dep_add'] = df.loc[i, 'dep_add'].where(df.loc[i, 'date_rev'] != df.loc[i, 'date'], df.loc[i, 'base'])
    
    ## 최종적으로 기초잔고액의 입금을 다 반영한 입금액을 계산한다
    df['dep_add'] = df['dep_add'].where(df['dep_add'] >= 0, 0)
    df['dep_mod'] = df['dep'] - df['dep_add']
    df['dep'] = df['dep_mod']
    
    ## 계산 중간에 발생한 임시열을 삭제한다
    df = df.drop(['base', 'end', 'base_temp', 'dep_cum', 'dep_cum_diff', 'dep_add', 'dep_mod'], axis=1)

    ## 매출(rev)이 음수(-)인 인덱스를 찾고, 해당 금액을 입금으로 반영한다. 그리고 매출은 0으로 변경한다.
    index_temp = df[df['rev'] < 0].index
    df.loc[index_temp, 'dep'] = df.loc[index_temp, 'dep'] - df.loc[index_temp, 'rev']
    df.loc[index_temp, 'rev'] = 0

    ## 입금(rev)이 음수(-)인 인덱스를 찾고, 해당 금액을 매출으로 반영한다. 그리고 입금은 0으로 변경한다.
    index_temp = df[df['dep'] < 0].index
    df.loc[index_temp, 'rev'] = df.loc[index_temp, 'rev'] - df.loc[index_temp, 'dep']
    df.loc[index_temp, 'dep'] = 0

    ## 기준일자의 매출액은 rev_temp에 저장한다
    df['rev_temp'] = df.loc[df_index, 'rev']
    df['rev_temp'] = df['rev_temp'].fillna(method='ffill')

    ## 입금(rev)에 대한 누적합 및 누적합의 차분(differentiate)를 구한다)
    df['dep_cum'] = df.groupby(['id'])['dep'].apply(lambda x: np.cumsum(x))
    df['dep_cum_diff'] = df.groupby(['id'])['dep_cum'].apply(lambda x: x.diff()).fillna(0)

    ## 기준일자 매출액을 입금 누적합과 비교하여, 매출액의 입금을 월별로 확인한다.
    df['dep_mod'] = df['dep'].where(df['rev_temp'] >= df['dep_cum'], df['rev_temp'] - (df['dep_cum'] - df['dep_cum_diff']))

    i = df.loc[df['dep_mod'] < 0].index
    df.loc[i, 'dep_mod'] = df.loc[i, 'dep_mod'].where(df.loc[i, 'date_rev'] != df.loc[i, 'date'], df.loc[i, 'rev'])
    df['dep_mod'] = df['dep_mod'].where(df['dep_mod'] >=0, 0)
    df = df.drop(['rev', 'dep', 'dep_cum', 'dep_cum_diff'], axis=1) # 필요없는 열을 삭제

    ## dueDateMonth : 매출발생 이후, 입금일 (단위는 1달 기준)을 입금된 날짜와 비교하여, 각 입금월의 연체기간을 계산한다
    for dueDateMonth_d in df['dueDateMonth'].unique():
        i = df.loc[df['dueDateMonth'] == dueDateMonth_d].index

        k = df.loc[i, 'date_rev'] + timedelta(days=30)*dueDateMonth_d - timedelta(days=10)
        k = k.apply(lambda x: date(x.year, x.month, calendar.monthrange(x.year, x.month)[1]))

        df.loc[i, 'due_delay'] = (df.loc[i, 'date'] - pd.to_datetime(k)).apply(lambda x: x.days)
    
    ## 연체일자(due_delay / 사실상 1달 단위날짜가 나옴)가 음수(-)인 경우 0으로 변경
    df['due_delay'] = df['due_delay'].where(df['due_delay'] >= 0, 0)

    ## 연체일자(due_delay)를 기준으로 A-H로 분류된다. (A: 정상회입 / B: 1달이내 연체 / C: 2달이내 연체 ...)
    df['due_label'] = pd.cut(df['due_delay'], bins=[0, 25, 55, 85, 115, 145, 175, 195, np.inf], labels=list('ABCDEFGH'), include_lowest=True)

    ## 자료형을 문자형으로 변경한다
    df['due_label'] = df['due_label'].astype('str')
    
    ## ID와 날짜(date)를 기준으로 Pivot테이블을 구하고, 자료와 Join한다
    df_p = df.pivot_table(index=['id', 'date'], columns='due_label', values='dep_mod', aggfunc=np.sum).fillna(0).reset_index()
    df_temp = df[df['date'] == date1].drop(['date_rev', 'dep_mod', 'due_delay', 'due_label'], axis=1)
    df_temp = df_temp.merge(df_p, on='id', how='left')
    
    ## 요약자료를 누적한다
    recap = recap.append(df_temp)

C:\Anaconda3\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


### 상기 코드로 정리된 자료는 아래와 같다

In [44]:
recap.head()

,A,B,C,D,E,F,G,H,date_x,date_y,dueDateMonth,id,rev_temp
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016-01-31,2016-01-31,1.0,100001001,13815136.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016-01-31,2016-02-29,1.0,100001001,13815136.0
2,0.0,3614150.0,0.0,0.0,0.0,0.0,0.0,0.0,2016-01-31,2016-03-31,1.0,100001001,13815136.0
3,0.0,0.0,10200986.0,0.0,0.0,0.0,0.0,0.0,2016-01-31,2016-04-30,1.0,100001001,13815136.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016-01-31,2016-05-31,1.0,100001001,13815136.0


In [45]:
recap['dep_sum'] = recap[['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']].sum(axis=1) # 입금된 금액의 합계를 구한다

In [55]:
recap_agg = recap.groupby(['id', 'date_x']).agg({'rev_temp':'max', 'dep_sum':'sum' , 'A':'sum', 'B':'sum', 'C':'sum', 'D':'sum', 'E':'sum'
                                     , 'F':'sum', 'G':'sum', 'H':'sum'}).reset_index() # id와 날짜를 기준으로 Groupby

recap_agg['I'] = recap_agg['rev_temp'] - recap_agg[['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']].sum(axis=1) # 회입되지 못한 금액을 계산

recap_agg = recap_agg[recap_agg['rev_temp'] != 0] # 매출이 발생하지 않은 기간(월)은 제외

recap_agg.columns = ['id', '날짜', '매출', '회수합계', '원금회수(정상)', '원금회수(1개월)', '원금회수(2개월)', '원금회수(3개월)'
                     , '원금회수(4개월)', '원금회수(5개월)', '원금회수(6개월)', '원금회수(6개월초과)', '미회수'] # 해더 이름변경

In [58]:
recap_agg.head()

,id,날짜,매출,회수합계,원금회수(정상),원금회수(1개월),원금회수(2개월),원금회수(3개월),원금회수(4개월),원금회수(5개월),원금회수(6개월),원금회수(6개월초과),미회수
0,100001001,2016-01-31,13815136.0,13815136.0,0.0,3614150.0,10200986.0,0.0,0.0,0.0,0.0,0.0,0.0
1,100001001,2016-02-29,14605186.0,14605186.0,0.0,14605186.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,100001001,2016-03-31,34379059.0,34379059.0,21652919.0,12726140.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,100001001,2016-04-30,45321000.0,45321000.0,5762991.0,7500000.0,0.0,0.0,0.0,32058009.0,0.0,0.0,0.0
4,100001001,2016-05-31,32683340.0,32683340.0,0.0,0.0,0.0,0.0,32683340.0,0.0,0.0,0.0,0.0
